In [3]:
import numpy as np

In [4]:
a = np.ones((2,1))
a[1] = 2
a

array([[1.],
       [2.]])

In [5]:
a2 = np.ones((3,1)) *2
a2

array([[2.],
       [2.],
       [2.]])

In [6]:
b = np.ones(4)
b[1] = 2
b[2] = 3
b[3] = 4
b

array([1., 2., 3., 4.])

In [7]:
c = a*b
d = a2*b
c

array([[1., 2., 3., 4.],
       [2., 4., 6., 8.]])

In [8]:
e = np.random.uniform((-2/np.sqrt(4)), (2/np.sqrt(4)), 4* 7)
e = np.reshape(e, (4, 7))
np.matmul(d, e)

array([[-1.86971839, -1.55236281,  3.88931846,  6.69381908, -8.3680899 ,
         1.34960578, -6.43906814],
       [-1.86971839, -1.55236281,  3.88931846,  6.69381908, -8.3680899 ,
         1.34960578, -6.43906814],
       [-1.86971839, -1.55236281,  3.88931846,  6.69381908, -8.3680899 ,
         1.34960578, -6.43906814]])

In [9]:
d

array([[2., 4., 6., 8.],
       [2., 4., 6., 8.],
       [2., 4., 6., 8.]])

Figure out how to merge

In [10]:
#aint it
np.outer(c,d)

array([[ 2.,  4.,  6.,  8.,  2.,  4.,  6.,  8.,  2.,  4.,  6.,  8.],
       [ 4.,  8., 12., 16.,  4.,  8., 12., 16.,  4.,  8., 12., 16.],
       [ 6., 12., 18., 24.,  6., 12., 18., 24.,  6., 12., 18., 24.],
       [ 8., 16., 24., 32.,  8., 16., 24., 32.,  8., 16., 24., 32.],
       [ 4.,  8., 12., 16.,  4.,  8., 12., 16.,  4.,  8., 12., 16.],
       [ 8., 16., 24., 32.,  8., 16., 24., 32.,  8., 16., 24., 32.],
       [12., 24., 36., 48., 12., 24., 36., 48., 12., 24., 36., 48.],
       [16., 32., 48., 64., 16., 32., 48., 64., 16., 32., 48., 64.]])

In [11]:
#I think this is right?
res = np.einsum("ik,jk->kij",c,d)
res

array([[[ 2.,  2.,  2.],
        [ 4.,  4.,  4.]],

       [[ 8.,  8.,  8.],
        [16., 16., 16.]],

       [[18., 18., 18.],
        [36., 36., 36.]],

       [[32., 32., 32.],
        [64., 64., 64.]]])

In [12]:
np.shape(res)

(4, 2, 3)

In [13]:
import torch
from siren import Sine #pip install siren-torch

In [18]:
class coordx_net(torch.nn.Module):
    def __init__(self):
        super(coordx_net, self).__init__()
        self.x_first = torch.nn.Linear(1,16)
        self.y_first = torch.nn.Linear(1,16)
        self.premerge_parallels = torch.nn.Sequential(
            Sine(),
            torch.nn.Linear(16,16),
            Sine(),
            torch.nn.Linear(16,16),
            Sine()
        )
        
        self.postmerge = torch.nn.Sequential(
            torch.nn.Linear(16,32),
            Sine(),
            torch.nn.Linear(32,3),
            torch.nn.Sigmoid()
        )
    
    def merge(self,x,y):
        return torch.einsum("ik,jk->ijk",x,y)

    def forward(self, x, y):
        x1 = self.x_first(x)
        y1 = self.y_first(y)
        x2 = self.premerge_parallels(x1)
        y2 = self.premerge_parallels(y1)
        merged = self.merge(x2,y2)
        print(merged.shape)
        res = self.postmerge(merged)
        print(res.shape)
        return res
    

In [19]:
net = coordx_net()
x = torch.ones((2,1)) * 1
print("x", x)
print("sine", Sine()(x))
y = torch.ones((3,1)) * 2
print( "y", y)
merged = net(x, y)
merged

x tensor([[1.],
        [1.]])
sine tensor([[0.8415],
        [0.8415]])
y tensor([[2.],
        [2.],
        [2.]])
torch.Size([2, 3, 32])
torch.Size([2, 3, 3])


tensor([[[0.4964, 0.4576, 0.4647],
         [0.4964, 0.4576, 0.4647],
         [0.4964, 0.4576, 0.4647]],

        [[0.4964, 0.4576, 0.4647],
         [0.4964, 0.4576, 0.4647],
         [0.4964, 0.4576, 0.4647]]], grad_fn=<SigmoidBackward0>)

# Try to add R

In [103]:
class coordx_net(torch.nn.Module):
    def __init__(self, r):
        super().__init__()
        self.R = r
        hidden = 16
        print(f"R: {r}, Hidden Layers: {hidden}")
        premerge_hidden = 16 * self.R

        self.x_first = torch.nn.Linear(1, premerge_hidden)
        self.y_first = torch.nn.Linear(1, premerge_hidden)
        self.premerge_parallels = torch.nn.Sequential(
            Sine(),
            torch.nn.Linear(premerge_hidden, premerge_hidden),
            Sine(),
            torch.nn.Linear(premerge_hidden, premerge_hidden),
            Sine()
        )
        
        self.postmerge = torch.nn.Sequential(
            torch.nn.Linear(hidden, hidden),
            Sine(),
            torch.nn.Linear(hidden, 3),
            torch.nn.Sigmoid()
        )
    
    def merge(self,x,y):
        x3 = torch.einsum("ijk->ik", x)
        y3 = torch.einsum("ijk->ik", y)
        print(x3.shape, y3.shape)
        return torch.einsum("ik,jk->ijk", x3, y3)

    def forward(self, x, y):
        x1 = self.x_first(x)
        y1 = self.y_first(y)
        x2 = self.premerge_parallels(x1)
        y2 = self.premerge_parallels(y1)
        x2 = x2.reshape((x2.shape[0], self.R, int(x2.shape[1]/self.R)))
        y2 = y2.reshape((y2.shape[0], self.R, int(y2.shape[1]/self.R)))
        print(x2.shape, y2.shape)
        merged = self.merge(x2,y2)
        print(merged.shape)
        res = self.postmerge(merged)
        print(res.shape)
        return res
    

In [104]:
net = coordx_net(2)
x = torch.ones((2,1)) * 1
y = torch.ones((3,1)) * 2
res = net(x,y)

R: 2, Hidden Layers: 16
torch.Size([2, 2, 16]) torch.Size([3, 2, 16])
torch.Size([2, 16]) torch.Size([3, 16])
torch.Size([2, 3, 16])
torch.Size([2, 3, 3])


In [105]:
res

tensor([[[0.4504, 0.4924, 0.5276],
         [0.4504, 0.4924, 0.5276],
         [0.4504, 0.4924, 0.5276]],

        [[0.4504, 0.4924, 0.5276],
         [0.4504, 0.4924, 0.5276],
         [0.4504, 0.4924, 0.5276]]], grad_fn=<SigmoidBackward0>)